In [17]:
import setup  # notebooks/setup.py
setup.init(verbose = True)

Applied nest_asyncio patch for Jupyter compatibility
Changed working directory to: C:\Users\gepanago\PycharmProjects\stack-exchange\StackExchange-Statistics-Service\src


In [18]:
from datetime import datetime
from app.components.stackexchange import StackExchangeClient
import json
from pathlib import Path
MOCK = True
MOCK_DATA = Path('mock_data')

In [19]:
client = StackExchangeClient()
if not MOCK:
    ##
    start_date = datetime(2025, 6, 1)
    end_date = datetime(2025, 6, 5)
    ##
    since = int(start_date.timestamp())
    until = int(end_date.timestamp())
    answers = client.get_answers(since, until)
else:
    filename = "answers_2025-06-01-00-00-00_2025-06-05-00-00-00.json"
    full_path = MOCK_DATA / filename
    answers = json.loads(full_path.read_text(encoding='utf-8'))
print(f"Fetched {len(answers)} number of answers")


Fetched 2057 number of answers


### Save fetched answers
``` If MOCK is set as False, API response is saved in the mock_data directory.```

In [20]:
if not MOCK:
    filename = f"answers_{start_date_string}_{end_date_string}.json"
    full_path = MOCK_DATA / filename
    full_path.write_text(json.dumps(answers, indent=4), encoding='utf-8')
    print(f"File {filename} saved sucessfully.")

In [21]:
accepted_answers = [answer for answer in answers if answer.get('is_accepted') == True]

In [ ]:
print(len(accepted_answers))

In [ ]:
answer_ids = [answer.get('answer_id') for answer in answers]

In [ ]:
len(answer_ids)

In [ ]:
comments = client.get_comments(answer_ids)

In [ ]:
len(comments)

In [ ]:
comments[:10]

### Some edge cases

#### Empty 

In [23]:
# Example test for empty results
MOCK = False
empty_answers = client.get_answers(int(datetime(2000, 1, 1).timestamp()), int(datetime(2000, 1, 2).timestamp()))
assert len(empty_answers) == 0, "Expected no answers for this date range."

empty_comments = client.get_comments([])
assert len(empty_comments) == 0, "Expected no comments for empty answer IDs."

In [24]:

empty_answers

[]

In [25]:
empty_comments

[]

#### Invalid range

In [26]:
# Invalid date range
invalid_answers = client.get_answers(int(datetime(2025, 6, 5).timestamp()), int(datetime(2025, 6, 1).timestamp()))
assert len(invalid_answers) == 0, "Expected no answers for invalid date range."

# Invalid answer IDs
invalid_comments = client.get_comments([-1, 0, 999999999999])
assert len(invalid_comments) == 0, "Expected no comments for invalid answer IDs."

Error fetching data from StackExchange API, url:https://api.stackexchange.com/2.3/answers/-1;0;999999999999/comments, page: 1 : 400 Client Error: Bad Request for url: https://api.stackexchange.com/2.3/answers/-1;0;999999999999/comments?order=asc&sort=creation&site=stackoverflow&page=1


In [27]:
invalid_answers

[]

In [28]:
invalid_comments

[]